In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mlp_term_2_2025_kaggle_assignment_3_path = kagglehub.competition_download('mlp-term-2-2025-kaggle-assignment-3')

print('Data source import complete.')


# Problem Statement

The goal of this task is to predict the rating a customer would give to an eatery based on their textual review. Using a labeled training dataset containing customer reviews and their corresponding ratings, we aim to build a machine learning model that can generalize well to unseen reviews in the test set.

# Importing necessary libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Accessing Data Input

In [ ]:
train=pd.read_csv('/kaggle/input/mlp-term-2-2025-kaggle-assignment-3/train.csv')
test=pd.read_csv('/kaggle/input/mlp-term-2-2025-kaggle-assignment-3/test.csv')

In [ ]:
raw_test=pd.read_csv('/kaggle/input/mlp-term-2-2025-kaggle-assignment-3/test.csv')

In [ ]:
train.head()

In [ ]:
test.head(2)

In [ ]:
train.shape

# Identifying Data Types

In [ ]:
pd.DataFrame(train.dtypes).T

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
#rating is target column
train['rating'].unique() #Multi-Classification Problem

In [ ]:
import re

def convert_to_days(text):
    if pd.isna(text):
        return np.nan

    text = str(text).lower().strip()

    # Handle cases like "a year ago", "an hour ago"
    if 'a year' in text:
        return 365
    elif 'a month' in text:
        return 30
    elif 'a week' in text:
        return 7
    elif 'a day' in text or 'an hour' in text:
        return 1

    match = re.search(r'\d+', text)
    if match:
        num = int(match.group())
        if 'year' in text:
            return num * 365
        elif 'month' in text:
            return num * 30
        elif 'week' in text:
            return num * 7
        elif 'day' in text:
            return num
        elif 'hour' in text:
            return 1
    return np.nan

train['review_time'] = train['review_time'].apply(convert_to_days)
test['review_time'] = test['review_time'].apply(convert_to_days)


# Descriptive Statistics

In [ ]:
train.describe().T

# Identifying and Handling Missing Data

## Training Set

In [ ]:
train.columns = train.columns.str.strip() #Many column names have spaces before or after the name.

In [ ]:
train.isnull().sum()

In [ ]:
#median imputation

for col in ['latitude', 'longitude']:
    median_val = train[col].median()
    train[col]=train[col].fillna(median_val)

train.isnull().sum()

In [ ]:
train.shape

## Testing Set

In [ ]:
test.columns = test.columns.str.strip()

In [ ]:
test.isnull().sum()

In [ ]:
#In test dataset we can't drop values. So imputing them with median
test['latitude']=test['latitude'].fillna(train['latitude'].median())
test['longitude']=test['longitude'].fillna(train['longitude'].median())

In [ ]:
test.isnull().sum()

# Identifying and Handling Duplicates

## Training Set

In [ ]:
train.duplicated().sum()

In [ ]:
train=train.drop_duplicates()

## Testing Set

In [ ]:
test.duplicated().sum()

In [ ]:
test=test.drop_duplicates()

# Identifying and Handling Outliers

## Training Set

In [ ]:
train['rating_count'] = train['rating_count'].str.replace(',', '', regex=False).astype(int)

In [ ]:
numeric_cols = [col for col in train.select_dtypes(include=['int64', 'float64']).columns if col not in ['id','rating']]
outlier_counts={}
def outliers_count(train,cols):
    for col in cols:
        Q1 = train[col].quantile(0.25)
        Q3 = train[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outliers = train[(train[col] < lower) | (train[col] > upper)]
        if outliers.shape[0]>0:
            outlier_counts[col] = outliers.shape[0]
    return outlier_counts
result=outliers_count(train,numeric_cols)
print(f"The total number of outliers in columns:{result}")

In [ ]:
sns.boxplot(x=train['rating_count'])

In [ ]:
caps=dict() #to store the capping values that can be used in test dataset
def cap_outliers(train, col):
    Q1 = train[col].quantile(0.25)
    Q3 = train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    caps[col]=(lower,upper)
    train[col] = train[col].clip(lower=lower, upper=upper)

for col in ['rating_count']:
    cap_outliers(train, col)

## Testing Set

In [ ]:
test['rating_count'] = test['rating_count'].str.replace(',', '', regex=False).astype(int)

In [ ]:
#Handling the outliers in the testing dataset according to training dataset
for col, (lower_cap, upper_cap) in caps.items():
    test[col] = test[col].clip(lower=lower_cap, upper=upper_cap)

# Visual Representations and Key Insights

## 1. Distribution of Ratings

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='rating', data=train, palette="viridis")
plt.title("Distribution of Customer Ratings")
plt.show()

### Key Insight
Ratings are skewed toward extremes (1 and 5), showing strong polarization in customer feedback.

Rating 5 has the highest count, indicating a significant proportion of highly satisfied customers.

Rating 1 is the second most common, highlighting many customers with very poor experiences.

Middle ratings (2, 3, 4) are comparatively fewer, suggesting customers tend to express either very positive or very negative experiences, rather than moderate opinions.


This polarization suggests that customer experiences are not balanced — businesses may need to investigate the causes behind the large number of low (1) ratings despite the strong presence of high (5) ratings.

## 2. Review Length vs Rating

In [ ]:
sns.boxplot(
    x='rating',
    y=train['review'].apply(lambda x: len(str(x).split())),
    data=train,
    palette="coolwarm"
)
plt.title("Review Length vs Rating")
plt.show()

### Key Insight
**When customers are dissatisfied, they tend to write longer, detailed reviews (explaining problems, complaints, or issues). In contrast, satisfied customers often leave short and positive comments ("Great food!", "Loved it!", etc.).**

## 3. Top Words by Rating

In [ ]:
from collections import Counter

# top words for rating=1
words_low = " ".join(train[train['rating']==1]['review'].astype(str)).lower().split()
common_low = Counter(words_low).most_common(10)

# top words for rating=5
words_high = " ".join(train[train['rating']==5]['review'].astype(str)).lower().split()
common_high = Counter(words_high).most_common(10)

# DataFrame for plotting
low_df = pd.DataFrame(common_low, columns=["word", "count"])
high_df = pd.DataFrame(common_high, columns=["word", "count"])

# Plot
fig, axes = plt.subplots(1,2, figsize=(14,6))

sns.barplot(x="count", y="word", data=low_df, ax=axes[0], palette="Reds_r")
axes[0].set_title("Top Words in Rating 1 Reviews")

sns.barplot(x="count", y="word", data=high_df, ax=axes[1], palette="Greens_r")
axes[1].set_title("Top Words in Rating 5 Reviews")

plt.tight_layout()
plt.show()


### Key Insight
Both Rating 1 and 5 reviews show high counts of stopwords (“the”, “and”, “to”, etc.), which are common filler words and not sentiment-bearing.

In Rating 1 reviews, stopwords dominate, with no strong negative sentiment words standing out clearly.

In Rating 5 reviews, besides stopwords, clear positive words such as “excellent”, “good”, and “food” appear, highlighting customer satisfaction.

**Key takeaway: While stopwords occur frequently in all reviews, positive reviews are more likely to include explicit positive sentiment words, whereas negative reviews do not show equally strong negative terms.**

## 4. Frequent Words in Customer Reviews for Lowest vs Highest Ratings

In [ ]:
from wordcloud import WordCloud

# Combine all reviews with rating = 1
text_low = " ".join(train[train['rating']==1]['review'].astype(str))
text_high = " ".join(train[train['rating']==5]['review'].astype(str))

# Word cloud for rating=1
plt.figure(figsize=(12,6))
wc_low = WordCloud(width=800, height=400, background_color="white", colormap="Reds").generate(text_low)
plt.subplot(1,2,1)
plt.imshow(wc_low, interpolation="bilinear")
plt.axis("off")
plt.title("Common Words in Rating 1 Reviews")

# Word cloud for rating=5
wc_high = WordCloud(width=800, height=400, background_color="white", colormap="Greens").generate(text_high)
plt.subplot(1,2,2)
plt.imshow(wc_high, interpolation="bilinear")
plt.axis("off")
plt.title("Common Words in Rating 5 Reviews")

plt.show()


### Key Insight
Rating 1 (Negative Reviews, Red Word Cloud):

Frequent words: food, order, service, staff, location, never, bad, one.

Insight: Customers giving low ratings mostly complain about poor service, order mistakes, slow or wrong food delivery, and staff behavior. Words like never, terrible, worst indicate strong dissatisfaction.

Rating 5 (Positive Reviews, Green Word Cloud):

Frequent words: food, good, McDonald, great, fast, excellent, place, love.

Insight: Customers giving high ratings highlight good food quality, fast service, cleanliness, and overall satisfaction. Positive words like excellent, great, and nice dominate.

# More Preprocessing

In [ ]:
#DROPPING USELESS COLUMNS
# train['category'].unique() -->constant
# train['store_name'].value_counts() ---> seems only for McDonald's Store , again constant
train=train.drop(['id','category','store_name'],axis=1)
test=test.drop(['id','category','store_name'],axis=1)

In [ ]:
#Feature Engineering
store_avg = train.groupby('store_address')['rating'].mean()
train['store_avg_rating'] = train['store_address'].map(store_avg)

# Map to test
test['store_avg_rating'] = test['store_address'].map(store_avg)

# Fill missing values in test (stores not seen in train)
global_avg = train['rating'].mean()
test['store_avg_rating']=test['store_avg_rating'].fillna(global_avg)

In [ ]:
train=train.drop(columns=['store_address'])
test=test.drop(columns=['store_address'])

In [ ]:
# Log-transform rating_count
train['rating_count'] = np.log1p(train['rating_count'])
test['rating_count'] = np.log1p(test['rating_count'])

## Splitting the Dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Step 1: Split into features and target
X = train.drop(columns=['rating'])
y = train['rating']

# Step 2: Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
test.info()

# Scaling & Encoding

In [ ]:
# Install sentence-transformers if not already
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# --- Load a small, fast model ---
st_model = SentenceTransformer('all-MiniLM-L6-v2')  # 384-dim embeddings

# --- Generate embeddings ---
X_text_train = st_model.encode(X_train['review'].tolist(), batch_size=32, show_progress_bar=True)
X_text_val   = st_model.encode(X_val['review'].tolist(), batch_size=32, show_progress_bar=True)
X_text_test  = st_model.encode(test['review'].tolist(), batch_size=32, show_progress_bar=True)

print("Original embedding shape:", X_text_train.shape)

# --- Dimensionality reduction with PCA ---
pca = PCA(n_components=100, random_state=42)
X_text_train_pca = pca.fit_transform(X_text_train)
X_text_val_pca   = pca.transform(X_text_val)
X_text_test_pca  = pca.transform(X_text_test)

print("Reduced embedding shape:", X_text_train_pca.shape)


In [ ]:
from sklearn.preprocessing import StandardScaler

As all the columns in dataset are numeric, hence encodig not required here.

In [ ]:
from sklearn.preprocessing import StandardScaler

scale_cols = ['latitude', 'longitude', 'rating_count', 'review_time', 'store_avg_rating']

scaler = StandardScaler()

X_train_num = scaler.fit_transform(X_train[scale_cols])
X_val_num   = scaler.transform(X_val[scale_cols])
test_num  = scaler.transform(test[scale_cols])

### Combining

In [ ]:
X_train = np.hstack([X_train_num, X_text_train_pca])
X_val  = np.hstack([X_val_num, X_text_val_pca])
test = np.hstack([test_num, X_text_test_pca])

print("Final train shape:", X_train.shape)
print("Final val shape:", X_val.shape)      #sparse matrix

In [ ]:
print(X_train.shape, X_val.shape, test.shape)

# Model Building

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(solver='saga', max_iter=1000)
logreg.fit(X_train, y_train)
logreg_preds = logreg.predict(X_val)
logreg_acc = accuracy_score(y_val, logreg_preds)
print(f"Logistic Regression Accuracy: {logreg_acc:.4f}")

## SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(loss='log_loss', max_iter=1000)
sgd.fit(X_train, y_train)
sgd_preds = sgd.predict(X_val)
sgd_acc = accuracy_score(y_val, sgd_preds)
print(f"SGD Classifier Accuracy: {sgd_acc:.4f}")

## RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_base = RandomForestClassifier(n_estimators=300, n_jobs=-1)
rf_base.fit(X_train, y_train)
rf_preds = rf_base.predict(X_val)
rf_acc = accuracy_score(y_val, rf_preds)
print(f"Random Forest Accuracy: {rf_acc:.4f}")

## ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score


etc_base = ExtraTreesClassifier(n_estimators=300, n_jobs=-1, random_state=42)
etc_base.fit(X_train, y_train)

etc_preds = etc_base.predict(X_val)
etc_acc = accuracy_score(y_val, etc_preds)
print(f"ExtraTrees Accuracy: {etc_acc:.4f}")



## XGBClassifier

In [ ]:
from xgboost import XGBClassifier

y_train_xgb = y_train - 1
y_val_xgb = y_val - 1

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train_xgb)
xgb_preds = xgb.predict(X_val)
xgb_acc = accuracy_score(y_val_xgb, xgb_preds)
print(f"XGBoost Accuracy: {xgb_acc:.4f}")

## LGBMClassifier

In [ ]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(verbose=-1)
lgbm.fit(X_train, y_train)
lgbm_preds = lgbm.predict(X_val)
lgbm_acc = accuracy_score(y_val, lgbm_preds)
print(f"LightGBM Accuracy: {lgbm_acc:.4f}")

## HistGradientBoostingClassifier

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score


hgb = HistGradientBoostingClassifier(
    max_iter=100,
    learning_rate=0.05,
    max_depth=None,
    random_state=0
)
hgb.fit(X_train, y_train)
y_pred = hgb.predict(X_val)
print("HistGradientBoosting Accuracy:", accuracy_score(y_val, y_pred))


# Hyperparameter Tuning

## RandomForestClassifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0, n_jobs=-1)

param_dist = {
    "n_estimators": [200, 500, 1000],  # more trees → better stability
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]
}

rf_search = RandomizedSearchCV(
    rf, param_distributions=param_dist,
    n_iter=20, cv=3, scoring="accuracy", verbose=2, n_jobs=-1, random_state=0
)
rf_search.fit(X_train, y_train)

print("Best RF params:", rf_search.best_params_)
print("Best RF accuracy:", rf_search.best_score_)


In [ ]:
y_pred = rf_search.best_estimator_.predict(X_val)
val_acc = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", val_acc)

## XGBClassifier

In [ ]:
from xgboost import XGBClassifier

# ✅ Fix labels for XGB (shift so that classes start at 0)
y_train_xgb = y_train - y_train.min()
y_val_xgb   = y_val - y_val.min()

# Define XGB model
xgb = XGBClassifier(
    objective="multi:softmax",
    num_class=len(np.unique(y_train_xgb)),  # important to use fixed labels
    eval_metric="mlogloss",
    random_state=0,
    n_jobs=-1,
    use_label_encoder=False
)

# Parameter search space
param_dist = {
    "n_estimators": [200, 400],              # smaller search (avoid 1000 for now)
    "max_depth": [3, 5, 7],                  # avoid 10 (very slow + overfitting risk)
    "learning_rate": [0.05, 0.1],            # keep mid values (0.01 too slow, 0.2 risky)
    "subsample": [0.8, 1.0],                 # fewer options
    "colsample_bytree": [0.8, 1.0],          # fewer options
    "min_child_weight": [1, 3]               # fewer options
}
# Randomized search
xgb_search = RandomizedSearchCV(
    xgb,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring="accuracy",
    verbose=2,
    n_jobs=-1,
    random_state=0
)

# Fit with shifted labels
xgb_search.fit(X_train, y_train_xgb)

print("Best XGB params:", xgb_search.best_params_)
print("Best XGB CV accuracy:", xgb_search.best_score_)


y_pred = xgb_search.best_estimator_.predict(X_val)
val_acc = accuracy_score(y_val_xgb, y_pred)
print("Validation Accuracy:", val_acc)


## HistGradientBoostingClassifier

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=0)

param_dist = {
    "learning_rate": [0.05, 0.1],          # small, practical values
    "max_depth": [None, 10],               # keep shallow + default
    "max_iter": [200, 500],                # avoid 1000 (too slow)
    "min_samples_leaf": [20, 50],          # fewer choices
    "l2_regularization": [0.0, 0.1]        # light regularization search
}

hgb_search = RandomizedSearchCV(
    hgb, param_distributions=param_dist,
    n_iter=20, cv=3, scoring="accuracy", verbose=2, n_jobs=-1, random_state=0
)
hgb_search.fit(X_train, y_train)

print("Best HGB params:", hgb_search.best_params_)
print("Best HGB accuracy:", hgb_search.best_score_)


In [ ]:
y_pred = hgb_search.best_estimator_.predict(X_val)
val_acc = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", val_acc)

# Comparison of Model Performances

In [ ]:
# Dictionary to store results
results = {}

# --- Logistic Regression (already trained) ---
acc = accuracy_score(y_val, logreg.predict(X_val))
results["Logistic Regression"] = {"Val Accuracy": acc}

# --- SGD Classifier (already trained) ---
acc = accuracy_score(y_val, sgd.predict(X_val))
results["SGD Classifier"] = {"Val Accuracy": acc}

# --- rf Classifier (already trained) ---
acc = accuracy_score(y_val, rf_base.predict(X_val))
results["Random Forest (Baseline)"] = {"Val Accuracy": acc}

# --- Random Forest (from search) ---
rf_best = rf_search.best_estimator_
acc = accuracy_score(y_val, rf_best.predict(X_val))
results["Random Forest (Tuned)"] = {"Val Accuracy": acc}

# --- etc Classifier (already trained) ---
acc = accuracy_score(y_val, etc_base.predict(X_val))
results["Extra Trees (Baseline)"] = {"Val Accuracy": acc}


# --- XGBoost (from search) ---
xgb_best = xgb_search.best_estimator_
acc = accuracy_score(y_val_xgb, xgb_best.predict(X_val))
results["XGBoost (Tuned)"] = {"Val Accuracy": acc}

# --- HistGradientBoosting (from search) ---
hgb_best = hgb_search.best_estimator_
acc = accuracy_score(y_val, hgb_best.predict(X_val))
results["HistGB (Tuned)"] = {"Val Accuracy": acc}

# --- LightGBM (already trained) ---
acc = accuracy_score(y_val, lgbm.predict(X_val))
results["LightGBM"] = {"Val Accuracy": acc}

# --- Convert to DataFrame ---
results_df = pd.DataFrame(results).T.sort_values(by="Val Accuracy", ascending=False)
print(results_df)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Reset index so model names are a column
plot_df = results_df.reset_index().rename(columns={"index": "Model"})

plt.figure(figsize=(10, 6))
sns.barplot(
    data=plot_df,
    x="Model",
    y="Val Accuracy",
    palette="viridis"
)

# Add annotations on top of bars
for i, v in enumerate(plot_df["Val Accuracy"]):
    plt.text(i, v + 0.005, f"{v:.3f}", ha="center", fontsize=9)

plt.title("Model Comparison - Validation Accuracy", fontsize=14)
plt.ylabel("Validation Accuracy", fontsize=12)
plt.xlabel("Models", fontsize=12)
plt.xticks(rotation=45, ha="right", fontsize=10)
plt.ylim(0, 1)  # since accuracy lies in [0, 1]
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()


# Final Submission

In [ ]:
# # --- Get prediction probabilities from both models ---
# proba_rf_base = rf_base.predict_proba(test)
# proba_rf_best = rf_best.predict_proba(test)

# # Weighted average of probabilities
# final_avg_proba = (
#     0.55 * proba_rf_base +
#     0.45 * proba_rf_best
# )

# # Convert probabilities to class labels
# final_predictions = np.argmax(final_avg_proba, axis=1)

# # Shift labels from 0–4 → 1–5
# final_predictions = final_predictions + 1
# print("Unique predicted classes:", np.unique(final_predictions))
# # --- Create submission DataFrame ---
# submission_df = pd.DataFrame({
#     "id": raw_test["id"],
#     "rating": final_predictions
# })

# # Save CSV
# submission_df.to_csv("submission.csv", index=False)
# print("Submission file saved as submission.csv")


In [ ]:
# # Predict on test set
# y_test_pred = xgb_search.best_estimator_.predict(test)

# # Shift back labels if required
# y_test_pred = y_test_pred + y_train.min()

# # Save submission
# submission = pd.DataFrame({"id":  raw_test["id"], "rating": y_test_pred})
# submission.to_csv("submission.csv", index=False)


In [ ]:
# from sklearn.ensemble import VotingClassifier

# voting_clf = VotingClassifier(
#     estimators=[
#         ('xgb', xgb_search.best_estimator_),
#         ('hgb', hgb_search.best_estimator_)
#     ],
#     voting='soft',  # average predicted probabilities
#     n_jobs=-1
# )

# voting_clf.fit(X_train, y_train_xgb)

# y_pred_vote = voting_clf.predict(X_val)
# val_acc_vote = accuracy_score(y_val_xgb, y_pred_vote)
# print("Validation Accuracy (XGB+HGB blend):", val_acc_vote)


In [ ]:
# # Predict on test set
# y_test_vote = voting_clf.predict(test)

# # Shift back to original labels (e.g., 1–5 instead of 0–4)
# y_test_vote = y_test_vote + y_train.min()

# # Check unique classes predicted
# print("Unique predicted classes (test):", np.unique(y_test_vote))


In [ ]:
# --- Get probability predictions on validation set ---
proba_xgb = xgb_search.best_estimator_.predict_proba(X_val)
proba_hgb = hgb_search.best_estimator_.predict_proba(X_val)
proba_rf  = rf_search.best_estimator_.predict_proba(X_val)

# --- Weighted averaging (0.6, 0.3, 0.1) ---
final_proba = (
    0.6 * proba_xgb +
    0.3 * proba_hgb +
    0.1 * proba_rf
)

# Take class with highest probability
final_pred = np.argmax(final_proba, axis=1)

# ✅ Compare with validation labels
val_acc_weighted = accuracy_score(y_val_xgb, final_pred)
print("Validation Accuracy (Weighted Blend: XGB+HGB+RF):", val_acc_weighted)

# --- Also for test set ---
test_proba_xgb = xgb_search.best_estimator_.predict_proba(test)
test_proba_hgb = hgb_search.best_estimator_.predict_proba(test)
test_proba_rf  = rf_search.best_estimator_.predict_proba(test)

final_test_proba = (
    0.6 * test_proba_xgb +
    0.3 * test_proba_hgb +
    0.1 * test_proba_rf
)

final_predictions = np.argmax(final_test_proba, axis=1)

# Shift back labels (since we subtracted y.min earlier)
final_predictions += y_train.min()

print("Unique predicted classes:", np.unique(final_predictions))


In [ ]:
submission = pd.DataFrame({
    "id": raw_test["id"],   # replace with your test dataframe ID column name
    "rating": final_predictions
})

submission.to_csv("submission.csv", index=False)
print("Submission saved as submission.csv")


In [ ]:
# import os

# # Replace with the exact filename you want to delete
# file_to_delete = 'submission.csv'

# # Delete if it exists
# if os.path.exists(file_to_delete):
#     os.remove(file_to_delete)
#     print(f"Deleted '{file_to_delete}'")
# else:
#     print(f"File '{file_to_delete}' not found")